# Nutree Tutorial

Nutree organizes arbitrary object instances in an unobtrusive way. <br>
That means, we can add existing objects without having to derrive from a common 
base class or having them implement a specific protocol.

# Setup some sample classes and objects

In [10]:
import uuid


class Department:
    def __init__(self, name: str):
        self.guid = uuid.uuid4()
        self.name = name

    def __str__(self):
        return f"Department<{self.name}>"


class Person:
    def __init__(self, name: str, age: int):
        self.guid = uuid.uuid4()
        self.name = name
        self.age = age

    def __str__(self):
        return f"Person<{self.name} ({self.age})>"

Now create some instances

In [11]:
development_dep = Department("Development")
test__dep = Department("Test")
marketing_dep = Department("Marketing")

alice = Person("Alice", 25)
bob = Person("Bob", 35)
claire = Person("Claire", 45)
dave = Person("Dave", 55)

Let's organize these objects in a hierarchical structure:

In [12]:
from nutree import Tree

tree = Tree("Organization")

dev_node = tree.add(development_dep)
test_node = dev_node.add(test__dep)
mkt_node = tree.add(marketing_dep)

tree.add(alice)
dev_node.add(bob)
test_node.add(claire)
mkt_node.add(dave)

tree.print()

Tree<'Organization'>
├── <__main__.Department object at 0x106e6e690>
│   ├── <__main__.Department object at 0x106e6d640>
│   │   ╰── <__main__.Person object at 0x106e6e990>
│   ╰── <__main__.Person object at 0x10683b200>
├── <__main__.Department object at 0x106e6d7f0>
│   ╰── <__main__.Person object at 0x106e6c6b0>
╰── <__main__.Person object at 0x106e6e660>


Tree nodes store a reference to the object in the `node.data` attribute.

The nodes are formatted by the object's  `__repr__` implementation by default. <br>
We can overide ths by passing an f-string as `repr` argument:

In [13]:
tree.print(repr="{node.data}")

Tree<'Organization'>
├── Department<Development>
│   ├── Department<Test>
│   │   ╰── Person<Claire (45)>
│   ╰── Person<Bob (35)>
├── Department<Marketing>
│   ╰── Person<Dave (55)>
╰── Person<Alice (25)>


# Access Nodes
We can use the index syntax to get the node object for a given data object:

In [25]:
tree[alice]

Node<'Person<Alice (25)>', data_id=275672678>

In [29]:
assert tree[alice].data is alice, "nodes store objects in data attribute"
alice_guid = alice.guid

# Add 

# Iteration and Searching

There are multiple methods to iterate the tree.

In [24]:
res = []
for node in tree:
    res.append(node.data.name)
print(res)

['Development', 'Test', 'Claire', 'Bob', 'Marketing', 'Dave', 'Alice']


# Mutation

# Data IDs and Clones

In [14]:
tree.print(repr="{node}", title=False)

Node<'Department<Development>', data_id=275672681>
├── Node<'Department<Test>', data_id=275672420>
│   ╰── Node<'Person<Claire (45)>', data_id=275672729>
╰── Node<'Person<Bob (35)>', data_id=275266336>
Node<'Department<Marketing>', data_id=275672447>
╰── Node<'Person<Dave (55)>', data_id=275672171>
Node<'Person<Alice (25)>', data_id=275672678>


# Serialization

In [15]:
tree.to_dict_list()

[{'data': 'Department<Development>',
  'children': [{'data': 'Department<Test>',
    'children': [{'data': 'Person<Claire (45)>'}]},
   {'data': 'Person<Bob (35)>'}]},
 {'data': 'Department<Marketing>',
  'children': [{'data': 'Person<Dave (55)>'}]},
 {'data': 'Person<Alice (25)>'}]

In [16]:
list(tree.to_list_iter())

[(0, {}), (1, {}), (2, {}), (1, {}), (0, {}), (5, {}), (0, {})]

In [20]:
t = Tree._from_list([(0, "A"), (0, "B"), (1, "C"), (0, "D"), (3, "E")])
print(t.format())

Tree<'4410903680'>
├── 'A'
│   ╰── 'C'
│       ╰── 'E'
├── 'B'
╰── 'D'
